In [125]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os
import surprise 
from sklearn.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from surprise import SVD
from IESEGRecSys import eval
from surprise import KNNBasic, SVD, CoClustering, Dataset, Reader

In [83]:
artist_df = pd.read_table('Data/Data Group Assignment/Artists.dat')
tags_df = pd.read_table('Data/Data Group Assignment/tags.dat',encoding='latin-1')
user_artists_df = pd.read_table('Data/Data Group Assignment/user_artists.dat')
user_taggedartists_df = pd.read_table('Data/Data Group Assignment/user_taggedartists.dat')


In [84]:
user_artists_df.head()

,userID,artistID,weight
0,2,51,13883
1,2,52,11690
2,2,53,11351
3,2,54,10300
4,2,55,8983


In [86]:
tags_df.head()

,tagID,tagValue
0,1,metal
1,2,alternative metal
2,3,goth rock
3,4,black metal
4,5,death metal


In [87]:
user_taggedartists_df

,userID,artistID,tagID,day,month,year
0,2,52,13,1,4,2009
1,2,52,15,1,4,2009
2,2,52,18,1,4,2009
3,2,52,21,1,4,2009
4,2,52,41,1,4,2009
...,...,...,...,...,...,...
186474,2100,16437,4,1,7,2010
186475,2100,16437,292,1,5,2010
186476,2100,16437,2087,1,7,2010
186477,2100,16437,2801,1,5,2010


In [88]:
#merging the dataset for content-based 
df_content_based = pd.merge(user_taggedartists_df,tags_df,on="tagID",how="left")

In [89]:
#number of genres
df_content_based["tagValue"].nunique()

9749

In [90]:
# Creating dummy variables:
df_content_based_1 = pd.get_dummies(df_content_based,prefix='', prefix_sep="" , columns=['tagValue'])
df_content_based_1 = df_content_based_1.drop(['userID', 'artistID','day','month','year'], axis=1)
df_content_based_1 = df_content_based_1.set_index("tagid")
df_content_based_1.head()

,userID,artistID,tagID,day,month,year,'80s,-pearl fashion music,0 play yet,00,...,zikirli,zmiel pierogi,zmierzch,zombie,zombie rave,zombieland,zoocore,zornish,ztt,zu
0,2,52,13,1,4,2009,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,52,15,1,4,2009,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,52,18,1,4,2009,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,52,21,1,4,2009,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,52,41,1,4,2009,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
#discretize weights using qcuts 
user_artists_df['weight_quantiles'] = pd.qcut(user_artists_df['weight'],
                           q=[0,.2,.4,.6,.8,1],
                           labels=False,
                           precision=0)
user_artists_df=user_artists_df.drop(["weight"], axis = 1)

In [93]:
# train-test split
train, test = train_test_split(user_artists_df, test_size=0.3, random_state=123)

# reset index
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

print(train.shape)
print(test.shape)

(64983, 3)
(27851, 3)


In [94]:
reader = surprise.Reader(rating_scale=(1, 5))
df_train = surprise.Dataset.load_from_df(train, reader).build_full_trainset()
df_test = list(test.itertuples(index=False, name=None))

In [95]:
# user-based
options = {'name':'cosine', 'user_based':True}
ub = KNNBasic(k=15, min_k=5, sim_options=options, random_state=42)

# item-based
options = {'name':'cosine', 'user_based':False}
ib = KNNBasic(k=15, min_k=5, sim_options=options, random_state=42)

# svd
mf = SVD(n_factors=20, biased=False, random_state=42)

# clustering
clust = CoClustering(n_cltr_u=10, n_cltr_i=10, n_epochs=50, random_state=42)

models = {"UB_15":ub, "IB_15":ib, "SVD_20":mf, "Clust_10_10":clust}
overview = pd.concat([eval.evaluate(mod.fit(df_train).test(df_test), topn=5, rating_cutoff=4) for mod in models.values()], axis=1)
overview.columns = list(models.keys())
overview

Computing the cosine similarity matrix...


C:\tools\Anaconda3\envs\py\lib\site-packages\surprise\prediction_algorithms\algo_base.py:249: RuntimeWarning: invalid value encountered in double_scalars
  sim = construction_func[name](*args)


Done computing similarity matrix.
Computing the cosine similarity matrix...


C:\tools\Anaconda3\envs\py\lib\site-packages\surprise\prediction_algorithms\algo_base.py:249: RuntimeWarning: invalid value encountered in double_scalars
  sim = construction_func[name](*args)


Done computing similarity matrix.


<ipython-input-95-10ebd00b95bb>:16: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  overview = pd.concat([eval.evaluate(mod.fit(df_train).test(df_test), topn=5, rating_cutoff=4) for mod in models.values()], axis=1)


,UB_15,IB_15,SVD_20,Clust_10_10
RMSE,1.393631,1.117180,1.217409,1.074960
MAE,1.183930,0.897713,0.968265,0.864092
Recall,0.000180,0.134892,0.080216,0.156115
Precision,0.250000,0.976562,0.867704,0.745704
F1,0.000359,0.237042,0.146855,0.258180
NDCG@5,0.866736,0.861017,0.868794,0.860111


In [132]:
print(len(df_content_based_1.columns))
print(len(pd.unique(df_content_based_1.columns)))

9748
9748


In [139]:
df_content_based_1

,'80s,-pearl fashion music,0 play yet,00,00's,007,00s,00s rock,1,1008,...,zikirli,zmiel pierogi,zmierzch,zombie,zombie rave,zombieland,zoocore,zornish,ztt,zu
tagID,,,,,,,,,,,,,,,,,,,,,
13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
292,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2087,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [138]:
df_content_based_1.groupby('tagID').count().

,'80s,-pearl fashion music,0 play yet,00,00's,007,00s,00s rock,1,1008,...,zikirli,zmiel pierogi,zmierzch,zombie,zombie rave,zombieland,zoocore,zornish,ztt,zu
tagID,,,,,,,,,,,,,,,,,,,,,
1,1729,1729,1729,1729,1729,1729,1729,1729,1729,1729,...,1729,1729,1729,1729,1729,1729,1729,1729,1729,1729
2,212,212,212,212,212,212,212,212,212,212,...,212,212,212,212,212,212,212,212,212,212
3,22,22,22,22,22,22,22,22,22,22,...,22,22,22,22,22,22,22,22,22,22
4,301,301,301,301,301,301,301,301,301,301,...,301,301,301,301,301,301,301,301,301,301
5,582,582,582,582,582,582,582,582,582,582,...,582,582,582,582,582,582,582,582,582,582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12643,7,7,7,7,7,7,7,7,7,7,...,7,7,7,7,7,7,7,7,7,7
12644,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
12645,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4


In [116]:
# init content-based
cb = ContentBased(NN=10)

# fit on content
cb.fit(df_content_based_1)

# fit on train_ratings
cb.fit_ratings(df_train)
    
# predict test ratings
cb_pred = cb.test(df_test)

AssertionError: Duplicate items in content data!

In [ ]:
# compute metrics
cb_res = eval.evaluate(cb_pred, topn=5, rating_cutoff=4).rename(columns={'value':'Content_based_10'})
cb_res